### 作業目標: 了解 N-Gram 如何藉由文本計算機率

### 為何需要使用馬可夫假設來簡化語言模型的計算?

原本的語言模型利用貝氏定理計算機率時為:
$$
\begin{aligned}
&W = (W_1W_2W_3W_4…W_m) \\
&P(W_1,W_2,W_3,W_4,…,W_m) = P(W_1)*P(W_2|W_1)*P(W_3|W_1,W_2)*…*P(W_m|W_1,…,W_{m-1})
\end{aligned}
$$

為何需要引入馬可夫假設使機率簡化為:
$$
P(W_m|W_1,W_2,W_3,…,W_{m-1}) = P(W_m|W_{m-n+1},W_{m-n+2},…,W_{m-1})
$$

In [3]:
'''
馬可夫模型為一種具有狀態的隨機過程，並用 P("下一狀態 s"|"當前狀態 S") 表示目前狀態轉移到下一個狀態的機率，
換句煥說，在 S 的前提下 s 發生的機率  

而語言模型利用貝式定理計算機率時，可透過上述公式表示。也可以發現其概念與馬可夫模型相似，
亦即，下一個字與當前 N-gram 合理的機率 (𝑃(𝑊𝑚|𝑊1, ..., 𝑊𝑚−1) = 𝑃(𝑊𝑚|𝑊𝑚−𝑛+1, 𝑊𝑚−𝑛+2, ...,𝑊𝑚−1))。
因此在計算 N-Gram 語言模型時，可以透過引入馬可夫模型來使機率簡化。
'''

'\n馬可夫模型為一種具有狀態的隨機過程，並用 P("下一狀態 s"|"當前狀態 S") 表示目前狀態轉移到下一個狀態的機率，\n換句煥說，在 S 的前提下 s 發生的機率  \n\n而語言模型利用貝式定理計算機率時，可透過上述公式表示。也可以發現其概念與馬可夫模型相似，\n亦即，下一個字與當前 N-gram 合理的機率 (𝑃(𝑊𝑚|𝑊1, ..., 𝑊𝑚−1) = 𝑃(𝑊𝑚|𝑊𝑚−𝑛+1, 𝑊𝑚−𝑛+2, ...,𝑊𝑚−1))。\n因此在計算 N-Gram 語言模型時，可以透過引入馬可夫模型來使機率簡化。\n'

### 以 Bigram 模型下判斷語句是否合理

已知的機率值有
1. p(i|_start_) = 0.25
2. p(english|want) = 0.0011
3. p(food|english) = 0.5
4. p(_end_|food) = 0.68
5. P(want|_start_) = 0.25
6. P(english|i) = 0.0011

In [2]:
import numpy as np
import pandas as pd
words = ['i', 'want', 'to', 'eat', 'chinese', 'food', 'lunch', 'spend']
word_cnts = np.array([2533, 927, 2417, 746, 158, 1093, 341, 278]).reshape(1, -1)
df_word_cnts = pd.DataFrame(word_cnts, columns=words)
df_word_cnts

,i,want,to,eat,chinese,food,lunch,spend
0,2533,927,2417,746,158,1093,341,278


In [3]:
# 記錄當前字與前一個字詞的存在頻率
bigram_word_cnts = [[5, 827, 0, 9, 0, 0, 0, 2], [2, 0, 608, 1, 6, 6, 5, 1], [2, 0, 4, 686, 2, 0, 6, 211],
                    [0, 0, 2, 0, 16, 2, 42, 0], [1, 0, 0, 0, 0, 82, 1, 0], [15, 0, 15, 0, 1, 4, 0, 0],
                    [2, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0]]

df_bigram_word_cnts = pd.DataFrame(bigram_word_cnts, columns=words, index=words)
df_bigram_word_cnts

,i,want,to,eat,chinese,food,lunch,spend
i,5,827,0,9,0,0,0,2
want,2,0,608,1,6,6,5,1
to,2,0,4,686,2,0,6,211
eat,0,0,2,0,16,2,42,0
chinese,1,0,0,0,0,82,1,0
food,15,0,15,0,1,4,0,0
lunch,2,0,0,0,0,1,0,0
spend,1,0,1,0,0,0,0,0


由上表可知當前一個字詞 (列, row index) 是 want 的時候, 當前字詞 (行, column name) 是 to 的頻率在文本中共有 608 次

In [8]:
# 請根據給出的總詞頻 (df_word_cnts) 與 bigram 模型的詞頻 (df_bigram_word_cnts) 計算出所有詞的配對機率 (ex:p(want|i))
df_bigram_prob = df_bigram_word_cnts.copy()

df_bigram_prob = df_bigram_prob / word_cnts.T

df_bigram_prob

,i,want,to,eat,chinese,food,lunch,spend
i,0.001974,0.32649,0.000000,0.003553,0.000000,0.000000,0.000000,0.000790
want,0.002157,0.00000,0.655879,0.001079,0.006472,0.006472,0.005394,0.001079
to,0.000827,0.00000,0.001655,0.283823,0.000827,0.000000,0.002482,0.087298
eat,0.000000,0.00000,0.002681,0.000000,0.021448,0.002681,0.056300,0.000000
chinese,0.006329,0.00000,0.000000,0.000000,0.000000,0.518987,0.006329,0.000000
food,0.013724,0.00000,0.013724,0.000000,0.000915,0.003660,0.000000,0.000000
lunch,0.005865,0.00000,0.000000,0.000000,0.000000,0.002933,0.000000,0.000000
spend,0.003597,0.00000,0.003597,0.000000,0.000000,0.000000,0.000000,0.000000


請根據已給的機率與所計算出的機率 (df_bigram_prob), 試著判斷下列兩個句子哪個較為合理

s1 = “i want english food”

s2 = "want i english food"

In [ ]:
'''
s1 = p(i|start) * p(want|i) * p(english|want) * p(food|english) * p(end|food)
   = 0.25 * p(want|i) * 0.0011 * 0.5 * 0.68
s2 = p(want|start) * p(i|want) * p(english|i) * p(food|english) * p(end|food)
   = 0.25 * p(i|want) * 0.0011 * 0.5 * 0.68

由上述算是中可看出，s1 和 s2 主要差在 p(want|i) 與 p(i|want)，而此兩機率題目皆未定義，
因此，若 p(want|i) 較大則 s1 較合理；而若 p(i|want) 較大則 s2 較合理。
'''